In [2]:
import requests
from bs4 import BeautifulSoup
import json
from openai import OpenAI

In [2]:
!pip install feedparser
import feedparser




  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=175ba23a7244a67d283b35a1529473760ef7f4730bc949b21a8caf7dd6b85e2d
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [12]:
url = "https://www.bbc.com/news/rss.xml"

feed = feedparser.parse(url)

for entry in feed.entries[:50]:
  print(entry.title)
  print(entry.link)
  print(entry.summary)
  print("\n")

Starmer apologises to Epstein victims for believing Mandelson's 'lies'
https://www.bbc.com/news/articles/cp37v4kyv3eo?at_medium=RSS&at_campaign=rss
The PM says the depth of the pair's relationship was not known when he was appointed US ambassador.


Could this be the beginning of the end for Starmer?
https://www.bbc.com/news/articles/c338g8n334mo?at_medium=RSS&at_campaign=rss
Few Labour MPs are publicly calling for Sir Keir Starmer to go but this is very serious moment for the PM.


Trump backs Chagos handover deal, says No 10
https://www.bbc.com/news/articles/c5yx1wedww2o?at_medium=RSS&at_campaign=rss
It comes after the US president suggested last month he could withdraw his support for the deal.


We had sex in a Chinese hotel, then found we had been broadcast to thousands
https://www.bbc.com/news/articles/c62rexy9y3no?at_medium=RSS&at_campaign=rss
A couple who stayed in Shenzhen discovered their intimate moments were filmed as spy-cam porn.


Number of homeless refugees in England s

In [13]:
articles_data = []

for entry in feed.entries[:10]:

  title = entry.title
  link = entry.link

  #fetch the actual article text
  article = requests.get(link).text
  soup = BeautifulSoup(article, "html.parser")

  paragraphs = soup.find_all("p")
  full_text = " ".join([p.get_text() for p in paragraphs])

  articles_data.append({
        "title": title,
        "link": link,
        "text": full_text
  })


  print(title)
  print(link)
  print(full_text)
  print("\n")

with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(articles_data, f, ensure_ascii=False, indent=2)


Starmer apologises to Epstein victims for believing Mandelson's 'lies'
https://www.bbc.com/news/articles/cp37v4kyv3eo?at_medium=RSS&at_campaign=rss
Sir Keir Starmer has apologised to the victims of Jeffrey Epstein for believing Lord Mandelson's "lies" and appointing him as the UK's ambassador to the US. The prime minister began a planned speech on funding to improve local communities by addressing the growing anger over how he had dealt with the issue. While he acknowledged it had been publicly known for some time that Lord Mandelson knew Epstein, he said: "None of us knew the depths and the darkness of that relationship." However, Conservative leader Kemi Badenoch said the PM's position was "untenable", while the Liberal Democrats called for a confidence vote to see if Labour MPs supported him. The prime minister is facing calls from some of his own MPs to stand down.  While the handful of backbenchers to do so publicly are frequent critics of the PM, many more have raised concerns pr

In [3]:
client = OpenAI(api_key = 'YOUR_KEY')



In [17]:
with open("articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)



In [20]:
for article in articles:
  content = article["text"]
  title = article["title"]


In [21]:
for article in articles:
  content = article["text"]
  summary = client.chat.completions.create(
      model = "gpt-4",
      messages = [
          {"role": "system", "content": "Summarize the foolowing article briefly."},
          {"role": "user", "content": content}
      ],
      max_tokens = 150
  )

  summary_text = summary.choices[0].message.content
  article["summary"] = summary_text

  print(f"Title: {article['title']}")
  print("Summary:", summary.choices[0].message.content)
  print("-"*50)




Title: Starmer apologises to Epstein victims for believing Mandelson's 'lies'
Summary: Prime Minister Sir Keir Starmer has apologized to victims of Jeffrey Epstein for believing Lord Mandelson's "lies" and selecting him as the UK's ambassador to the US. Starmer is facing claims from his own Conservative party MPs to resign, while the Liberal Democrats have called for a confidence vote. The Labour Party's PM has promised to release documents showing Lord Mandelson lied about his relationship with Epstein during the vetting process for the ambassador role. After his speech, Starmer stated he intended to continue improving the country for the better.

--------------------------------------------------
Title: Could this be the beginning of the end for Starmer?
Summary: UK Prime Minister, Sir Keir Starmer, is faced with increasing pressure after admitting to believing lies about appointing Jeffrey Epstein's associate, Peter Mandelson, to the UK's top diplomatic post. MPs express their fury 

In [22]:
summaries = []
for article in articles:
    summaries.append({
        "title": article["title"],
        "summary": article["summary"]
    })

with open("summaries.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)